In [41]:
!pip install -r requirements.txt

3658.10s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable


In [43]:
import csv
from dataset import datasets
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [56]:
dataset = datasets.C100Dataset('./dataset/data/cifar100_nl.csv' './dataset/data/cifar100_nl.csv')
[data_nl_tr_x, data_nl_tr_y, data_nl_val_x, data_nl_val_y] = dataset.getDataset()

AttributeError: 'C100Dataset' object has no attribute 'printImage'

In [49]:
print ('/dataset/data/cifar100_nl.csv')

/dataset/data/cifar100_nl.csv
